In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/mhmmd-nauman/DataSets/master/black-box-models/CP-Nets/dsbc/original%20ds/breast-cancer%20-%20complete.csv')
df = data
#del data['Unnamed: 32']

In [ ]:
index = data.index
number_of_rows = len(index)
print(number_of_rows)

277


In [ ]:
data.head()

,Age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,30-39,premeno,30-34,0-2,no,3,left,left_low,no,B
1,40-49,premeno,20-24,0-2,no,2,right,right_up,no,B
2,40-49,premeno,20-24,0-2,no,2,left,left_low,no,B
3,60-69,ge40,15-19,0-2,no,2,right,left_up,no,B
4,40-49,premeno,0-4,0-2,no,2,right,right_low,no,B


In [ ]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_X_age = LabelEncoder()
df['Age'] = labelencoder_X_age.fit_transform(df['Age'])
labelencoder_X_menopause = LabelEncoder()
df['menopause'] = labelencoder_X_menopause.fit_transform(df['menopause'])
labelencoder_X_tumor_size = LabelEncoder()
df['tumor-size'] = labelencoder_X_tumor_size.fit_transform(df['tumor-size'])
labelencoder_X_inv_nodes = LabelEncoder()
df['inv-nodes'] = labelencoder_X_inv_nodes.fit_transform(df['inv-nodes'])
labelencoder_X_node_caps = LabelEncoder()
df['node-caps'] = labelencoder_X_node_caps.fit_transform(df['node-caps'])
labelencoder_X_deg_malig = LabelEncoder()
df['deg-malig'] = labelencoder_X_deg_malig.fit_transform(df['deg-malig'])
labelencoder_X_breast = LabelEncoder()
df['breast'] = labelencoder_X_breast.fit_transform(df['breast'])
labelencoder_X_breast_quad = LabelEncoder()
df['breast-quad'] = labelencoder_X_breast_quad.fit_transform(df['breast-quad'])
labelencoder_X_irradiat = LabelEncoder()
df['irradiat'] = labelencoder_X_irradiat.fit_transform(df['irradiat'])
labelencoder_X_class = LabelEncoder()
df['Class'] = labelencoder_X_class.fit_transform(df['Class'])

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
df = clean_dataset(df)

In [ ]:
X = data.iloc[:, 0:9].values
y = data['Class']

In [ ]:
X

array([[ 1,  2,  5, ...,  0,  1,  0],
       [ 2,  2,  3, ...,  1,  4,  0],
       [ 2,  2,  3, ...,  0,  1,  0],
       ...,
       [ 4,  0,  8, ...,  1,  4,  1],
       [ 3,  2, 10, ...,  1,  2,  0],
       [ 2,  2,  5, ...,  1,  2,  0]])

In [ ]:
y

0      0
1      0
2      0
3      0
4      0
      ..
272    1
273    1
274    1
275    1
276    1
Name: Class, Length: 277, dtype: int64

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
y

0      0
1      0
2      0
3      0
4      0
      ..
272    1
273    1
274    1
275    1
276    1
Name: Class, Length: 277, dtype: int64

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
def create_baseline(x_size, y_size):
    # create model
    model = Sequential()
    model.add(Dense(36, input_dim=x_size, kernel_initializer='normal', activation='relu'))
    model.add(Dense(x_size,  activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer. mean_squared_error
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Initialising the ANN
#classifier = Sequential()
model = create_baseline(X.shape[1], 1)

In [ ]:
# Adding the input layer and the first hidden layer
#classifier.add(Dense(3, activation='relu', input_dim=5))
# Adding dropout to prevent overfitting
#classifier.add(Dropout(p=0.1))

In [ ]:
# Adding the output layer
#classifier.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compiling the ANN
#classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Fitting the ANN to the Training set
#classifier.fit(X_train, y_train, batch_size=10, epochs=50)
# Long scroll ahead but worth
# The batch size and number of epochs have been set using trial and error. Still looking for more efficient ways. Open to suggestions.
model.fit(X_train, y_train,
    batch_size=20,
    epochs=150,
    #shuffle=True,
    verbose=0 # Change it to 2, if wished to observe execution
    #validation_data=(kc_x_valid, kc_y_valid)
    ) 

In [ ]:
# Predicting the Test set results
#y_pred = classifier.predict(X_test)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
#score = classifier.evaluate(X_test,y_test)
score = model.evaluate(X_test,y_test)
print(score)

3/3 [==============================] - 0s 4ms/step - loss: 0.1939 - accuracy: 0.7976
[0.1939285397529602, 0.7976190447807312]


In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
Sp = tn / (tn+fp)
Sn = tp/(tp + fn)

In [ ]:
print(cm)
print(Sp)
print(Sn)

[[50  6]
 [11 17]]
0.8928571428571429
0.6071428571428571


In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_score = cohen_kappa_score(y_test, y_pred)
print(cohen_score)

0.5233644859813084
